In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

In [2]:
# 드라이버 설정
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
driver = webdriver.Chrome(options=options)

In [3]:
# 카테고리 리스트
categories = {
    "스킨/토너": "100000100010013",
    "에센스/세럼/앰플": "100000100010014",
    "크림": "100000100010015"
}

In [4]:
def get_all_products(category_id):
    url = f"https://www.oliveyoung.co.kr/store/display/getCategoryShop.do?dispCatNo={category_id}"
    driver.get(url)
    time.sleep(2)  # 로딩 대기
    
    # 무한 스크롤
    last_height = driver.execute_script("return document.body.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        # 상품 리스트 추출
    product_elements = driver.find_elements(By.CSS_SELECTOR, "ul.cate_prd_list li")
    product_links = []
    for elem in product_elements:
        try:
            link = elem.find_element(By.CSS_SELECTOR, "a").get_attribute("href")
            product_links.append(link)
        except:
            continue
    return product_links


In [5]:
def get_reviews(product_url):
    driver.get(product_url)
    time.sleep(2)
    
    # 리뷰 탭 클릭
    try:
        review_tab = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CLASS_NAME, "goods_reputation"))
        )
        review_tab.click()
    except:
        print(f"리뷰 탭 없음: {product_url}")
        return []

    time.sleep(2)
    reviews = []

    while True:
        time.sleep(1)
        review_items = driver.find_elements(By.CSS_SELECTOR, "#gdasList li")
        for item in review_items:
            try:
                text = item.find_element(By.CSS_SELECTOR, ".review_cont .txt_inner").text.strip()
                point = item.find_element(By.CSS_SELECTOR, ".review_cont .point").get_attribute("style")
                reviews.append({"text": text, "point": point})
            except:
                continue
                  # 다음 페이지 클릭
        try:
            next_button = driver.find_element(By.CSS_SELECTOR, ".pageing a.next")
            next_button.click()
            time.sleep(2)
        except:
            break

    return reviews

In [6]:
# 전체 실행
for category_name, category_id in categories.items():
    print(f"카테고리: {category_name}")
    product_links = get_all_products(category_id)
    print(f"상품 개수: {len(product_links)}개")

    for link in product_links:
        print(f"상품 링크: {link}")
        reviews = get_reviews(link)
        print(f"리뷰 개수: {len(reviews)}")
        for review in reviews:
            print(f"점수: {review['point']}, 내용: {review['text']}")


카테고리: 스킨/토너
상품 개수: 20개
상품 링크: https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000222334&dispCatNo=100000100010013&trackingCd=Cat100000100010013_Clickbest&curation=like&egcode=c001_c001&rccode=pc_category_01_a&egrankcode=1&t_page=%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EA%B4%80&t_click=%EB%A7%8E%EC%9D%B4%EB%B3%B8%EC%83%81%ED%92%88&t_number=1
리뷰 개수: 100
점수: width: 100%;, 내용: 향이 강하다고 해서 걱정했는데 저한테는 그렇게 역한 향도 아니고향 자체가 은은 해서 좋은 것 같아요. 그리고 속건조 진짜 제대로 해결 됩니다.😀
점수: width: 100%;, 내용: 친구대신 구매한 제품인데 건성인 분들에게 이만한 제품이 없다고 너무 잘 사용하는 중이라고 하더라구요!!
점수: width: 100%;, 내용: 촉촉하고 순해요 원플러스원이라 팍팍 써도 오래 쓸 수 있습니다
점수: width: 100%;, 내용: 여름에 가볍기 사용하기 좋은 토너예요 향이 있지만 거슬릴 정도의 향은 아니고 은은하게 나요 수부지에게 추천합니다
점수: width: 100%;, 내용: 제 동생이 피부가 예민한 편이라 바르라고 사준건데 이거 바르고 그나마 피부에 올라오던게 가라앉아서 꾸준히 잘 사용하고 있다고 합니다!
점수: width: 100%;, 내용: 바이오더마 클렌징워터 쓰고 좋아서 하이드라비오 토너랑 에센스 중에 고민하다가 지성이라 토너로 구매했는데 겨울철에는 속보습까지 완전 채워주지는 않지만 가벼운 제형으로 지성이 쓰기 좋고 다가올 여름철에도 부담없이 쓸 수 있을 거 같아요!! 제형이 가벼워서 저는 안개분사 미스트에다가 소분해서 쓰는데 수시로 뿌려주기 좋아용
점수: 

KeyboardInterrupt: 